In [1]:
import pandas as pd
import os.path as op
import os

In [56]:
dfile = "../data/mg_template.csv"
outdir = "../data/out/"

In [110]:
df = pd.read_csv(dfile)

In [112]:
df['name'] = [op.basename(i).split(".")[0] for i in df['mg_f']]
df

,mg_f,mg_r,wgs_technology,join,name
0,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,illumina,True,SRS019030
1,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,illumina,True,SRS022524
2,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,illumina,True,SRS078197
3,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,NaN,pyro,False,SRR063559
4,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,NaN,pyro,False,SRR063903
5,/mnt/stepanauskas_nfs/julia/testfragrecruitmen...,NaN,pyro,False,SRR061736
6,pairedexample/in/a/directory/pe.fastq,NaN,illumina,True,pe


In [115]:
mglist = list(df.mg_f[df['join']==False])
mglist

['/mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/pyro_mgs/SRS019030_454/SRR063559.fastq.gz',
 '/mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/pyro_mgs/SRS022524_454/SRR063903.fastq.gz',
 '/mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/pyro_mgs/SRS078197_454/SRR061736.fastq.gz']

In [116]:
tojoin = df.loc[df['join']==True]

In [117]:
list(tojoin.mg_f)
list(tojoin.mg_r)

['/mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/illumina_mgs/SRS019030/SRS019030.denovo_duplicates_marked.trimmed.2.fastq.gz',
 '/mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/illumina_mgs/SRS022524/SRS022524.denovo_duplicates_marked.trimmed.2.fastq.gz',
 '/mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/illumina_mgs/SRS078197/SRS078197.denovo_duplicates_marked.trimmed.2.fastq.gz',
 nan]

In [118]:

for n, f, r in zip(tojoin['name'], tojoin.mg_f, tojoin.mg_r):
    prefix = op.basename(f).split(".")[0]
    outdir = op.dirname(f)
    print(prefix, outdir, n)
    #if pd.isnull(r):
    #    r=None
    #else:
        #joinedfq = join(prefix, f, fq2=r, threads=cores, mmd=mmd, mino=mino, maxo=maxo, threads=threads, outdir=outdir)
        #mglist.append(joinedfq)

SRS019030 /mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/illumina_mgs/SRS019030 SRS019030
SRS022524 /mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/illumina_mgs/SRS022524 SRS022524
SRS078197 /mnt/stepanauskas_nfs/julia/testfragrecruitment/hmp/illumina_mgs/SRS078197 SRS078197
pe pairedexample/in/a/directory pe


In [ ]:
def process_multi_mgs(intable, outdir, threads, mmd, mino, maxo, threads, outdir, minlen):
    
    df = pd.read_csv(intable)
    df.name = [op.basename(i).split(".")[0] for i in df['mg_f']]
    mglist = list(df.mg_f[df['join']==False])
    
    # join reads identified as joined 
    tojoin = df.loc[df['join']==True]
    
    for n, f, r in zip(tojoin['name'], tojoin['mg_f'], tojoin['mg_r']:
        if pd.isnull(r):  # if reverse read cell is blank, but join=True, reads assumed to be interleaved
            r=None
        joinedfq = join(n, f, fq2=r, threads=cores, mmd=mmd, mino=mino, maxo=maxo, threads=threads, outdir=outdir)
        mglist.append(joinedfq)
    
    # size fileter all reads:
    processed_mgs = []
    read_counts = []
    nameorder = []
    for m in mglist:
        nameorder.append(op.basename(m).split(".")[0])
        newfilename = op.basename(m).replace(".fastq", "minlen_%s.fastq" % minlen)
        outfile = op.join(outdir, newfilename)
        new_out, count = read_size_filter(m, minlen, outfile, cores=threads)
        
        processed_mgs.append(new_out)
        read_counts.append(count)
                       
    data = {'name':nameorder, 'to_recruit': processed_mgs, 'read_count':read_counts}
    newinfo = pd.DataFrame(data)
    
    res_table = pd.merge(df, newinfo, how='outer', on='name')
    return res_table